In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import torch
from torch import nn

from hparams.hparams_GT import create_hparams
from datasets import LJDataset, TextMelCollate

In [2]:
!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Sat Mar 13 19:09:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.04    Driver Version: 455.23.04    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:19:00.0 Off |                  N/A |
| 75%   66C    P2   261W / 370W |  11673MiB / 24268MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:68:00.0 Off |                  N/A |
| 68%   

In [3]:
from parallel_wavegan.utils import download_pretrained_model
from parallel_wavegan.utils import load_model

device='cuda'
vocoder_tag = "ljspeech_parallel_wavegan.v3"

# NOTE: Sometimes download is failed due to "Permission denied". That is 
#   the limitation of google drive. Please retry after serveral hours.
vocoder = load_model(download_pretrained_model(vocoder_tag)).to(device).eval()
vocoder.remove_weight_norm()
print('done')

from parallel_wavegan.utils import read_hdf5

stats_filename = "/home/scpark/.cache/parallel_wavegan/ljspeech_parallel_wavegan.v3/stats.h5"
mean = read_hdf5(stats_filename, 'mean')
scale = read_hdf5(stats_filename, 'scale')
print('done')

done
done


In [4]:
stt_hparams, tts_hparams = create_hparams()
testset = LJDataset(tts_hparams, split='test')
collate_fn = TextMelCollate(tts_hparams)

test_loader = torch.utils.data.DataLoader(testset, num_workers=1, 
                          shuffle=False, sampler=None, batch_size=1, pin_memory=False,
                          drop_last=True, collate_fn=collate_fn)
print(test_loader)

In [5]:
def to_cuda(batch):
    batch['text'] = batch['text'].cuda()
    batch['text_lengths'] = batch['text_lengths'].cuda()
    batch['mels'] = batch['mels'].cuda()
    batch['mel_lengths'] = batch['mel_lengths'].cuda()
    
    return batch

In [51]:
import os
from os import listdir
from os.path import isfile, isdir, join, getsize

root = '/home/scpark/hard/datasets/LibriTTS/train-clean-100/'
#!ls $root
dirs = ['103', '150', '200', '250']
dirs = [join(root, f) for f in dirs if isdir(join(root, f))]

sub_dirs = []
for dir in dirs:
    sub_dirs += [join(dir, f) for f in listdir(dir) if isdir(join(dir, f))]

files = []
for dir in sub_dirs:
    files += [join(dir, f) for f in listdir(dir) if isfile(join(dir, f)) and '.normalized.txt' in f]
    
print(len(files))

for file in files:
    with open(file, 'r') as f:
        l = f.readline()
        print(l)

486
"Oh, I'm so glad.
"It isn't heavy. I've got all my worldly goods in it, but it isn't heavy.
I wasn't a bit sick coming over in the boat.
"That's Barry's pond," said matthew.
Orchard Slope's the name of his place.
With this Matthew's companion stopped talking, partly because she was out of breath and partly because they had reached the buggy.
I never could find out.
And then I can imagine that I'm dressed gorgeously.
"Well now, I'm afraid I can't," said matthew, who was getting a little dizzy.
And I've never had a pretty dress in my life that I can remember-but of course it's all the more to look forward to, isn't it?
Oh, mr Cuthbert!!
"Well now, I just can't recollect that I ever had."
I never expected I would, though.
"Well now, I dunno," said matthew.
"It's a boy I've come for.
"I'm very glad to see you.
Maybe they were out of boys of the brand you wanted."
They had simply rounded a curve in the road and found themselves in the "Avenue."
Because, you see, if the bridge DID crumpl

"Never mind," continued the captain; "I will have him yet before I'm a major."
He viewed the figure of Captain Wharton, as he stood proudly swelling with a pride that disdained further concealment, and exclaimed with great earnestness,--
"No, mr Caesar, running is not my trade." While speaking, he walked deliberately to the window, where the family were already collected in the greatest consternation.
The ladies left the table to their guests, who proceeded, without much superfluous diffidence, to do proper honors to the hospitality of mr Wharton.
On reaching the road which led through the bottom of the valley, they turned their horses' heads to the north.
As nothing could be seen likely to interrupt the enjoyments and harmony of such a day, the sisters descended to the parlor, with a returning confidence in their brother's security, and their own happiness.
Nothing remained now, but to meet the impending examination with as much indifference as the family could assume.
At length Capta

His sister was less delicate, and directed her eyes towards mr Darcy with a very expressive smile.
She was very equal, therefore, to address mr Bingley on the subject of the ball, and abruptly reminded him of his promise; adding, that it would be the most shameful thing in the world if he did not keep it.
Bingley met them with hopes that mrs Bennet had not found Miss Bennet worse than she expected.
They have at least that advantage."
She performed her part indeed without much graciousness, but mrs Bennet was satisfied, and soon afterwards ordered her carriage.
"Removed!" cried Bingley.
"Whatever I do is done in a hurry," replied he; "and therefore if I should resolve to quit Netherfield, I should probably be off in five minutes.
She would not listen, therefore, to her daughter's proposal of being carried home; neither did the apothecary, who arrived about the same time, think it at all advisable.
"I did not know before," continued Bingley immediately, "that you were a studier of charac

It was the latter part of July when Margaret returned home.
Coach building is a trade all the same, and I think a much more useless one than that of butchers or bakers.
He had a trick of half opening his mouth as if to speak, which constantly unsettled the form of the lips, and gave the face an undecided expression.
Helstone was at some distance from any neighbours of their own standard of cultivation.
The fern harvest was over, and now that the rain was gone, many a deep glade was accessible, into which Margaret had only peeped in July and August weather.
She was so happy out of doors, at her father's side, that she almost danced; and with the soft violence of the west wind behind her, as she crossed some heath, she seemed to be borne onwards, as lightly and easily as the fallen leaf that was wafted along by the autumnal breeze.
I think we are far better off, knowing only cottagers and labourers, and people without pretence.'
He would sigh aloud as he answered, that if he could do wha

In [16]:
# !mkdir inference

In [ ]:
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import time 
import soundfile as sf

inference_dir = 'inference/GT/'
!mkdir $inference_dir

vocoder.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        print(i, batch['strings'][0])
        batch = to_cuda(batch)
        _s = batch['mels'].data.cpu().numpy()
        _mel = _s[0].T
        _mel = (_mel - mean) / scale
        mel = torch.from_numpy(_mel).cuda()
        wav = vocoder.inference(mel)
        _wav = wav[:, 0].data.cpu().numpy()
        save_path = inference_dir + 'inference_' + str(i) + '.wav'
        sf.write(save_path, _wav, 22050, subtype='PCM_16')
        
print('done')

mkdir: cannot create directory ‘inference/GT/’: File exists
0 The Chronicles of Newgate, Volume two. By Arthur Griffiths. Section five: Newgate down to eighteen eighteen, part two.
1 The squalor and uncleanness of the debtors' side was intensified by constant overcrowding.
2 Prisoners were committed to it quite without reference to its capacity.
3 No remonstrance was attended to,
4 no steps taken to reduce the number of committals, and the governor was obliged to utilize the chapel as a day and night room.
5 Besides this, although the families of debtors were no longer permitted to live with them inside the jail,
6 hundreds of women and children came in every morning to spend the day there, and there was no limitation whatever to the numbers of visitors admitted to the debtors' side.
7 Friends arrived about nine a.m., and went out at nine p.m., when as many as two hundred visitors have been observed leaving the debtors' yards at one time.
8 The day passed in revelry and drunkenness. Al

78 Mr. Bennet refers to a gentleman confined for want of bail, who occupied a room with five others
79 two committed by the Bankruptcy Commissioner, one for perjury, and two transports.
80 Persons convicted of publishing libels were still immured in the same rooms with transports and felons.
81 The middle yard, as far as its limits would permit, was appropriated to felons and transports. The wards here were generally very crowded.
82 Each ward was calculated to hold twenty-four, allowing each individual one foot and a half;
83 Quote, a common-sized man, says the keeper, Mr. Newman, can turn in nineteen inches, end quote.
84 These twenty-four could just sleep on the barrack bed; when the number was higher, and it often rose to forty, the surplus had to sleep on the floor.
85 The crowding was in consequence of the delay in removing transports.
86 These often remained in Newgate for six months, sometimes a year, in some cases longer;
87 in one, for seven years -- that of a man sentenced t

158 To constitute this the aristocratic quarter, unwarrantable demands were made upon the space properly allotted to the female felons,
159 and no lodger was rejected, whatever his status, who offered himself and could bring grist to the mill.
160 The luxury of the state side was for a long time open to all who could pay
161 the convicted felon, the transport awaiting removal, the lunatic whose case was still undecided,
162 the misdemeanant tried or untried, the debtor who wished to avoid the discomfort of the crowded debtors' side, the outspoken newspaper editor,
163 or the daring reporter of parliamentary debates.
164 The better class of inmate complained bitterly of this enforced companionship with the vile,
165 association at one time forbidden by custom, but which greed and rapacity long made the rule.
166 The fee for admission to the state side, as fixed by the table of fees, was three guineas, but Mr. Newman declared that he never took more than two.
167 Ten and sixpence a week 